# IML4.1 Construção de mecanismos de captação de dados

**Título da Atividade**:  Construção de mecanismos de captação de dados

**Orientações**: Elabore um mecanismo de extração de dados capaz de obter os dados de uma página do IMDB. Por exemplo, crie um mecanismo computacional capaz de realizar Web Scrapping da página do filme Interestelar no IMDB (ou seja, https://www.imdb.com/title/tt0816692/) e armazene todos os dados obtidos em um arquivo do tipo ".csv". Sendo que, é importante que a solução proposta siga as boas práticas de programação, ou seja tenha Design Patterns, não tenha importações que não estão sendo utilizadas, o código esteja documentado, etc.

**Entrega**: Ao término da atividade, o trabalho deve ser submetido no ambiente de ensino como um arquivo python ou jupyter notebook (ou seja, arquivos com a extensão .py ou ipynb), no qual deve conter todas as etapas para a captação dos dados necessários. Além disso, os códigos gerados devem seguir os princípios de boas práticas de programação.

**Solução**: Foi criado um mecanismo de extração de dados com web scraping para a páginas do filme "Interestelar", mas também incluí as URLs dos filmes "Ex-Machina" e "Aniquilação" do IMDB, foram utilizadas as bibliotecas `requests` e `BeautifulSoup`, que são comuns para web scraping. O código seguirá as boas práticas de programação, incluindo documentação e evitando importações desnecessárias.

In [32]:
import requests
from bs4 import BeautifulSoup
import csv
import json


def scrape_imdb(url):
    """
    Função para realizar web scraping na página do IMDB de um filme específico.
    
    Args:
    url (str): URL da página do IMDB do filme.
    
    Returns:
    dict: Dicionário contendo os dados extraídos do filme.
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    full_title = soup.find('meta', property='og:title')['content']
    title = full_title.split('(')[0].strip()

    description = soup.find('meta', attrs={'name': 'description'})['content']

    rating = full_title.split('⭐')[1].split('|')[0].strip() if '⭐' in full_title else 'N/A'

    genre_script = soup.select_one('script[type="application/ld+json"]').contents[0]
    genre_data = json.loads(genre_script)
    genre_list = genre_data.get('genre', [])
    genre = ', '.join(genre_list) if isinstance(genre_list, list) else genre_list
    
    return {
        'Title': title,
        'Description': description,
        'Rating': rating,
        'Genre': genre
    }


def save_to_csv(data, filename):
    """
    Função para salvar dados em um arquivo CSV.

    Args:
    data (list): Lista de dicionários com os dados a serem salvos.
    filename (str): Nome do arquivo CSV.
    """
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=data[0].keys())
        writer.writeheader()
        for movie in data:
            writer.writerow(movie)

In [33]:
# Lista de URLs
urls = [
    'https://www.imdb.com/title/tt0816692/',
    'https://www.imdb.com/title/tt0470752/',
    'https://www.imdb.com/title/tt2798920/'
]

# Lista para armazenar os dados dos filmes
movies_data = []

# Percorrendo as URLs e coletando os dados
for url in urls:
    movie_data = scrape_imdb(url)
    movies_data.append(movie_data)

# Salvando os dados em um arquivo CSV
save_to_csv(movies_data, 'imdb_data.csv')
